In [1]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import utils
from utils import SessionStatus, classification_accuracy
from network import NeuralNetwork

In [2]:
# args parser
#------------------
# config contains all model and session setup options
P = utils.Parser()
my_args = '-a softplus'
config = P.parse_args_from_jupyter_notebook(my_args)
P.print_args()

SESSION CONFIG
  data_path: ./data/
       seed: 12345
 model_name: {}_{}
  name_suff: 
 activation: <class 'functions.SoftPlus'>
    dropout: False
  optimizer: <class 'optimizers.SGD'>
  objective: <class 'functions.SoftmaxCrossEntropy'>
   channels: [4, 64, 3]
 learn_rate: 0.01
  num_iters: 2000
 batch_size: 6
    verbose: False




In [3]:
#==============================================================================
# Model, session config
#==============================================================================
# Model config
#------------------
channels   = config.channels
activation = config.activation
optimizer  = config.optimizer
objective  = config.objective
dropout    = config.dropout

# Session config
#------------------
num_iters  = config.num_iters
batch_size = config.batch_size
verbose = config.verbose

In [4]:
trainer = utils.Trainer(channels, optimizer, objective, activation, num_iters, batch_size)

In [5]:
trainer()

#==============================================================================
# Model Summary: 

NeuralNetwork
  Layers: 
     1 : Dense (4, 64)
          : SoftPlus
     2 : Dense (64, 3)

- OPTIMIZER : SGD
- OBJECTIVE : SoftmaxCrossEntropy

# Training results, 2000 iterations
#------------------------------
            Error  |  Accuracy
* Average: 0.23019 | 0.94944
*  Median: 0.21888 | 1.00000
#------------------------------

# Test results, 30 samples
#------------------------------
            Error  |  Accuracy
* Average: 0.11398 | 0.96667
*  Median: 0.04486 | 1.00000
#------------------------------
#==============================================================================


In [6]:
def get_results_by_act():
    

In [7]:
trainer.session_status.train_history

array([[1.0850916 , 0.6666667 ],
       [1.105326  , 0.33333334],
       [1.1609786 , 0.        ],
       ...,
       [0.2731461 , 0.8333333 ],
       [0.08262651, 1.        ],
       [0.13079728, 1.        ]], dtype=float32)

In [13]:
str(activation())

'SoftPlus'